# Network optimization for revenue management
## Part 1 - Linear programming 

#### Dependencies

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import shutil
import sys
import os.path



import pyomo.environ as pyomo



#### Data loading

In [3]:
#Read data tables
products=pd.read_csv('products.csv', sep=';')
m=len(products)
flights=pd.read_csv('flights.csv', sep=';')
n=len(flights)
aircrafts=pd.read_csv('aircrafts.csv', sep=';')
p=len(aircrafts)
print('Read : ',m,' products ',n,' flights ',p,' aircrafts ')

Read :  30  products  6  flights  6  aircrafts 


#### List of all products, their associated price (=fare in €), the current number of seats sold and the demand forecast

In [4]:
products

,Route,BookingClass,Name,Fare,SeatsSold,ExpectedDemandToCome,ListofFlights
0,FCOAMS,High,'FCOAMS_High',650,5,5,"[AF1005,AF1040]"
1,FCOLHR,High,'FCOLHR_High',600,0,0,"[AF1005,AF1080]"
2,BCNAMS,High,'BCNAMS_High',550,5,10,"[AF1049,AF1040]"
3,FCOFRA,High,'FCOFRA_High',550,5,10,"[AF1005,AF1018]"
4,BCNFRA,High,'BCNFRA_High',525,5,5,"[AF1049,AF1018]"
5,BCNLHR,High,'BCNLHR_High',500,0,0,"[AF1049,AF1018]"
6,MADAMS,High,'MADAMS_High',450,5,5,"[AF1001,AF1040]"
7,MADFRA,High,'MADFRA_High',425,10,10,"[AF1001,AF1018]"
8,MADLHR,High,'MADLHR_High',400,0,5,"[AF1001,AF1080]"
9,FCOCDG,High,'FCOCDG_High',350,5,0,[AF1005]


#### List of all scheduled flights, their origin and destination, and the physical aircraft ID currently used

In [5]:
flights

,FlightNumber,FlightDeparture,FlightArrival,AircraftId
0,AF1001,MAD,CDG,1
1,AF1049,BCN,CDG,2
2,AF1005,FCO,CDG,3
3,AF1080,CDG,LHR,4
4,AF1018,CDG,FRA,5
5,AF1040,CDG,AMS,6


#### List of all the aircrafts in the fleet and their capacity

In [6]:
aircrafts

,AircraftId,Capacity
0,1,50
1,2,80
2,3,100
3,4,50
4,5,80
5,6,100


In [10]:
A=np.zeros(shape=(n,m))

for i in range(n):           
    flight = flights['FlightNumber'][i]
    Ori = flights['FlightDeparture'][i]
    Dst = flights['FlightArrival'][i]
    print('i = ',i," creating constraint for flight : ", flight," ",Ori,Dst)

    for j in range(m):
        list_flights = products['ListofFlights'][j]       
        if(flight in list_flights):
            print('   Match flight with product : ',products['Name'][j])
            A[i,j] = 1

i =  0  creating constraint for flight :  AF1001   MAD CDG
   Match flight with product :  'MADAMS_High'
   Match flight with product :  'MADFRA_High'
   Match flight with product :  'MADLHR_High'
   Match flight with product :  'MADCDG_High'
   Match flight with product :  'MADFRA_Low'
   Match flight with product :  'MADLHR_Low'
   Match flight with product :  'MADAMS_Low'
   Match flight with product :  'MADCDG_Low'
i =  1  creating constraint for flight :  AF1049   BCN CDG
   Match flight with product :  'BCNAMS_High'
   Match flight with product :  'BCNFRA_High'
   Match flight with product :  'BCNLHR_High'
   Match flight with product :  'BCNCDG_High'
   Match flight with product :  'BCNAMS_Low'
   Match flight with product :  'BCNFRA_Low'
   Match flight with product :  'BCNCDG_Low'
   Match flight with product :  'BCNLHR_Low'
i =  2  creating constraint for flight :  AF1005   FCO CDG
   Match flight with product :  'FCOAMS_High'
   Match flight with product :  'FCOLHR_High'
   

In [8]:

print(A)

[[0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  1. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 0. 1. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 1. 0. 0. 1. 0.]]


### Create vector of remaining capacity

In [9]:
C = np.zeros(n)
for i in range(n):
    Capacity = aircrafts['Capacity'][i]
    for j in range (m):
        Capacity -= products['SeatsSold'][j]*A[i,j]
    C[i] = Capacity
    print(Capacity)

5.0
20.0
45.0
15.0
10.0
40.0


C=[5,
20,
45,
15,
10,
40,
]

In [19]:
print("n = %d flights"%n)
print("m = %d products"%m)

n = 6 flights
m = 30 products


### Solving problem with pyomo

In [20]:

model = pyomo.ConcreteModel()


model.x = pyomo.Var(products['Name'], domain=pyomo.NonNegativeReals)


model.revenue = pyomo.Objective(sense = pyomo.maximize,\
                                expr = sum(model.x[products['Name'][i]]*\
                                   products['Fare'][i] for i in range(m)))

 

In [21]:

model.demand = pyomo.ConstraintList()


for i in range(m):
    model.demand.add(model.x[products['Name'][i]] \
                     <= products['ExpectedDemandToCome'][i])


model.capacity = pyomo.ConstraintList()


for i in range(n):
    flight = flights['FlightNumber'][i]
    print('i = ',i," creating constraint for flight : ", flight)
    model.capacity.add(sum([model.x[products['Name'][j]]*A[i,j] \
                            for j in range(m)]) <= C[i])


solver = pyomo.SolverFactory('glpk')


solver.solve(model)

print('Optimal allocation')
for i in range(m):
    print('  ', products['Name'][i], ':', model.x[products['Name'][i]](), 'demand accepted')
print()
print('revenue = €', model.revenue())       

Optimal allocation
   'FCOAMS_High' : 5.0 demand accepted
   'FCOLHR_High' : 0.0 demand accepted
   'BCNAMS_High' : 10.0 demand accepted
   'FCOFRA_High' : 10.0 demand accepted
   'BCNFRA_High' : 0.0 demand accepted
   'BCNLHR_High' : 0.0 demand accepted
   'MADAMS_High' : 0.0 demand accepted
   'MADFRA_High' : 0.0 demand accepted
   'MADLHR_High' : 5.0 demand accepted
   'FCOCDG_High' : 0.0 demand accepted
   'BCNCDG_High' : 5.0 demand accepted
   'CDGFRA_High' : 0.0 demand accepted
   'CDGLHR_High' : 5.0 demand accepted
   'MADCDG_High' : 0.0 demand accepted
   'CDGAMS_High' : 5.0 demand accepted
   'BCNAMS_Low' : 0.0 demand accepted
   'FCOLHR_Low' : 0.0 demand accepted
   'FCOFRA_Low' : 0.0 demand accepted
   'BCNFRA_Low' : 0.0 demand accepted
   'FCOAMS_Low' : 10.0 demand accepted
   'MADFRA_Low' : 0.0 demand accepted
   'BCNCDG_Low' : 5.0 demand accepted
   'FCOCDG_Low' : 5.0 demand accepted
   'CDGFRA_Low' : 0.0 demand accepted
   'MADLHR_Low' : 0.0 demand accepted
   'MADAMS_Lo

In [22]:
pyomo.SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 23000.0
  Upper bound: 23000.0
  Number of objectives: 1
  Number of constraints: 37
  Number of variables: 31
  Number of nonzeros: 79
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06613612174987793
# ----------------------------------------------------------
#   Solution Information
# ------------------------------